RDF programs

In [10]:
pip install rdflib

Note: you may need to restart the kernel to use updated packages.



[notice] A new release of pip is available: 23.2 -> 23.2.1
[notice] To update, run: python.exe -m pip install --upgrade pip


In [11]:
from rdflib import URIRef, BNode, Literal, Namespace, Graph
from rdflib.namespace import FOAF, DCTERMS, XSD, RDF, RDFS, SDO
import pprint
import os

In [12]:
# returns all proxies for a provided person
def linkedProxies(ProvidedPerson, g):
    query = """
    PREFIX rdf: <http://www.w3.org/1999/02/22-rdf-syntax-ns#>
    PREFIX ns2: <http://data.biographynet.nl/rdf/>
    PREFIX idm: <http://www.intavia.eu/idm-core/>
    select distinct ?s
    where { 
        ?s idm:person_proxy_for <%s> .
    }
    """ % ProvidedPerson
    
    proxies = g.query(query)
    return proxies

In [13]:
#returns all triples connected to proxy to depth of 2
def allTriples(Proxy, g): 
    # create a graph for a description
    # add all triples with subject that matches the description
    new_graph = Graph()
    new_graph += g.triples((Proxy, None, None))
    for stmt in g.triples((Proxy, None, None)):
        subject = stmt[2]
        new_graph += g.triples((subject, None, None))
    return new_graph

In [14]:
def deleteNewLine(s, p, o):
    if type(o) == Literal and "\n" in o:
        o = o.replace("\n", "")
        o = Literal(o)
    return s, p, o

In [25]:
# for one provided person, write all rdf statements into a named graph
def PersonToRDF(ProvidedPerson, g, new_g, path, new_file_path):
    for row in linkedProxies(ProvidedPerson, g):
        Proxy = row['s']
        all_triples = allTriples(Proxy, g)
        for triple in all_triples:
            new_g.add(triple)
    new_g.serialize(destination=new_file_path, format = 'trig')

In [26]:
#make a query to find all the persons in the knowledge graph
def find_persons(g):
    query = """
    PREFIX rdf: <http://www.w3.org/1999/02/22-rdf-syntax-ns#>
    PREFIX ns2: <http://data.biographynet.nl/rdf/>
    PREFIX idm: <http://www.intavia.eu/idm-core/>
    select distinct ?person
    where { 
        ?person rdf:type idm:Provided_Person .
    }
    """

    persons = g.query(query)
    return persons

In [27]:
def iterate_through_folder(folder_path, path):
    for root, dirs, files in os.walk(folder_path):
        for file_name in files:
            file_path = os.path.join(root, file_name)
            new_file_name = file_name.split('.')[0] + '.trig'
            new_file_path = os.path.join(path, new_file_name)
            #create a graph for the knowledge graph, for other knowledge graphs paste the file location and name within the parse function
            g = Graph()
            g.parse(file_path)
            new_g = Graph()
            Persons = find_persons(g)
            for row in Persons:
                Person = row['person']
                PersonToRDF(Person, g, new_g, path, new_file_path)

In [28]:
path = "D:\\bioport_xml_batch_15_10_rdf_graphs"
if not os.path.exists(path):
  os.mkdir(path)
  print("Folder %s created!" % path)
else:
  print("Folder %s already exists" % path)
folder_path = "D:\\bioport_xml_batch_15_10_graphs"
iterate_through_folder(folder_path, path)

Folder D:\bioport_xml_batch_15_10_rdf_graphs already exists


Failed to convert Literal lexical form to value. Datatype=http://www.w3.org/2001/XMLSchema#date, Converter=<function parse_date at 0x00000217E8045A80>
Traceback (most recent call last):
  File "C:\Users\cjtya\AppData\Local\Programs\Python\Python311\Lib\site-packages\rdflib\term.py", line 2084, in _castLexicalToPython
    return conv_func(lexical)  # type: ignore[arg-type]
           ^^^^^^^^^^^^^^^^^^
  File "C:\Users\cjtya\AppData\Local\Programs\Python\Python311\Lib\site-packages\isodate\isodates.py", line 201, in parse_date
    return date(sign * int(groups['year']),
           ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
ValueError: month must be in 1..12
